In [1]:
import pandas as pd

In [3]:
#####

# README: This code turns x-y coordinate files into x-y-z files that can be used by Ovito

#####

# Change the code here to your coordinate file and file paths

###

code = 'USW00023130'

###

s1 = 'Coords Folder/Weather Station Coords/Los Angeles/'

s2 = '_building_coords.txt'

with open(s1 + code + s2) as file:

    coords = pd.read_csv(file, sep = ',', names = ['Area', 'x', 'y'])

coords

,Area,x,y
0,3.216206e-08,-118.507032,34.154117
1,6.835830e-08,-118.505120,34.153723
2,1.557710e-08,-118.506413,34.152506
3,8.460220e-09,-118.507874,34.152790
4,8.063096e-09,-118.506971,34.153953
...,...,...,...
114800,1.564638e-08,-118.544532,34.218428
114801,4.432173e-09,-118.543341,34.187504
114802,1.859023e-09,-118.543829,34.187646
114803,1.520458e-08,-118.543829,34.187783


In [119]:
# Drop the class column from the frame

coords = coords.drop(['Area'], axis = 1)

# Create the normalized dataframe

normalized_data = pd.DataFrame()

measure = [[1, 0], [1, 0]]

index = 0

abs_max = 0

# Find the normalization factor for both x and y

for letter in ['x', 'y']:
    
    localMax = coords[letter].max()
    
    localMin = coords[letter].min()
    
    abs_local = max(abs(localMax), abs(localMin))
    
    if abs_local > abs_max:
        
        abs_max = abs_local
        
for letter in ['x', 'y']:
    
    normalized_data[letter] = coords[letter].divide(abs_max)
    
    mean = normalized_data[letter].mean()

    normalized_data[letter] = normalized_data[letter].subtract(mean)

    normalized_data[letter] = normalized_data[letter].multiply(10000)
    
    for scale in range(len(measure[index])): 
    
        num = measure[index][scale]
        
        num = num/10000 
    
        num = num + mean  
    
        num = num * abs_max  
        
        measure[index][scale] = num
        
    index += 1
    
scale = ((((measure[0][0] - measure[0][1])**2 + (measure[1][0] - measure[1][1])**2)/2)**(1/2))

# The multiplication factor here comes from the amount of feet in one degree of latitude. It's a rough measure

line = 'One degree of Ovito coords is about equal to ' + str(int(scale*364000)) + ' feet!'

print(line)

normalized_data

One degree of Ovito coords is about equal to 2692 feet!


,x,y
0,4.749296,-5.220362
1,4.022107,2.493342
2,-5.346514,-4.810769
3,-5.943876,0.493195
4,-5.668819,3.806856
...,...,...
193281,2.656266,5.523104
193282,-0.219558,-2.998040
193283,2.657140,5.531898
193284,-7.248378,-1.830536


In [120]:
normalized_data.insert(0, 'test', 'H')

normalized_data[''] = 0

normalized_data.loc[-1] = ['', '', '', '']

normalized_data.index = normalized_data.index + 1

normalized_data = normalized_data.sort_index()

r, c = normalized_data.shape

# Setting one of the 'z' values as 0.01 to prevent degeneracy (lol)

normalized_data.columns = ['1', '2', '3', '4']

normalized_data['4'][1] = 0.01

normalized_data.columns = [r - 1, '', '', '']

normalized_data

,193286,,,
0,,,,
1,H,4.7493,-5.22036,0.01
2,H,4.02211,2.49334,0
3,H,-5.34651,-4.81077,0
4,H,-5.94388,0.493195,0
...,...,...,...,...
193282,H,2.65627,5.5231,0
193283,H,-0.219558,-2.99804,0
193284,H,2.65714,5.5319,0
193285,H,-7.24838,-1.83054,0


In [121]:
# Found here: https://stackoverflow.com/questions/41428539/data-frame-to-file-txt-python/41428596

normalized_data.to_csv(path_or_buf = s1 + code + '_building_coords_reformatted.txt', index = False, header = True, sep = '\t', mode = 'a')

# Write the scale to a text file

with open(s1 + code + '_scale.txt', 'w+') as file:

    file.write(str(line))